<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 821ms/step - loss: 0.6819 - accuracy: 0.5414 - val_loss: 0.6957 - val_accuracy: 0.4341
Epoch 2/10
26/26 [==============================] - 20s 780ms/step - loss: 0.6635 - accuracy: 0.5900 - val_loss: 0.6637 - val_accuracy: 0.5659
Epoch 3/10
26/26 [==============================] - 20s 777ms/step - loss: 0.6326 - accuracy: 0.7129 - val_loss: 0.6279 - val_accuracy: 0.7171
Epoch 4/10
26/26 [==============================] - 20s 781ms/step - loss: 0.5887 - accuracy: 0.7543 - val_loss: 0.5765 - val_accuracy: 0.7610
Epoch 5/10
26/26 [==============================] - 20s 777ms/step - loss: 0.5430 - accuracy: 0.7774 - val_loss: 0.5410 - val_accuracy: 0.7317
Epoch 6/10
26/26 [==============================] - 20s 780ms/step - loss: 0.4809 - accuracy: 0.8041 - val_loss: 0.4794 - val_accuracy: 0.8244
Epoch 7/10
26/26 [==============================] - 20s 781ms/step - loss: 0.4200 - accuracy: 0.8370 - val_loss: 0.5202 - val_accuracy: 0.6732

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.67.h5
26/26 - 21s - loss: 0.6789 - accuracy: 0.5377 - val_loss: 0.6738 - val_accuracy: 0.5073
Epoch 2/5

Epoch 00002: saving model to weights.02-0.64.h5
26/26 - 21s - loss: 0.6507 - accuracy: 0.6350 - val_loss: 0.6394 - val_accuracy: 0.7854
Epoch 3/5

Epoch 00003: saving model to weights.03-0.60.h5
26/26 - 20s - loss: 0.6073 - accuracy: 0.7336 - val_loss: 0.5971 - val_accuracy: 0.7707
Epoch 4/5

Epoch 00004: saving model to weights.04-0.55.h5
26/26 - 20s - loss: 0.5536 - accuracy: 0.7713 - val_loss: 0.5477 - val_accuracy: 0.7756
Epoch 5/5

Epoch 00005: saving model to weights.05-0.46.h5
26/26 - 20s - loss: 0.4919 - accuracy: 0.8041 - val_loss: 0.4568 - val_accuracy: 0.7951


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 22s - loss: 0.6680 - accuracy: 0.6095 - val_loss: 0.6479 - val_accuracy: 0.5561


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6655 - accuracy: 0.6022 - val_loss: 0.6565 - val_accuracy: 0.5415
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.5968 - accuracy: 0.6934 - val_loss: 0.5561 - val_accuracy: 0.7366


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6637 - accuracy: 0.5742 - val_loss: 0.6317 - val_accuracy: 0.7805
Epoch 2/50
26/26 - 20s - loss: 0.6193 - accuracy: 0.6934 - val_loss: 0.5925 - val_accuracy: 0.6488
Epoch 3/50
26/26 - 20s - loss: 0.5482 - accuracy: 0.7470 - val_loss: 0.5847 - val_accuracy: 0.6537
Epoch 4/50
26/26 - 20s - loss: 0.5139 - accuracy: 0.7555 - val_loss: 0.4469 - val_accuracy: 0.8098
Epoch 5/50
26/26 - 20s - loss: 0.4555 - accuracy: 0.8102 - val_loss: 0.4385 - val_accuracy: 0.8439
Epoch 6/50
26/26 - 20s - loss: 0.4029 - accuracy: 0.8236 - val_loss: 0.3738 - val_accuracy: 0.8829
Epoch 7/50
26/26 - 20s - loss: 0.3663 - accuracy: 0.8601 - val_loss: 0.4112 - val_accuracy: 0.8439
Epoch 8/50
26/26 - 20s - loss: 0.2831 - accuracy: 0.9002 - val_loss: 0.2392 - val_accuracy: 0.9171
Epoch 9/50
26/26 - 20s - loss: 0.2582 - accuracy: 0.8966 - val_loss: 0.3018 - val_accuracy: 0.8390
Epoch 10/50
26/26 - 21s - loss: 0.2181 - accuracy: 0.9343 - val_loss: 0.1482 - val_accuracy: 0.9561
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6590 - accuracy: 0.5973 - val_loss: 0.6616 - val_accuracy: 0.5220
Epoch 2/5
26/26 [==============================] - 21s 790ms/step - loss: 0.5826 - accuracy: 0.7275 - val_loss: 0.5472 - val_accuracy: 0.7317
Epoch 3/5
26/26 [==============================] - 21s 792ms/step - loss: 0.5328 - accuracy: 0.7336 - val_loss: 0.5609 - val_accuracy: 0.6829
Epoch 4/5
26/26 [==============================] - 21s 789ms/step - loss: 0.4697 - accuracy: 0.7835 - val_loss: 0.4204 - val_accuracy: 0.7902
Epoch 5/5
26/26 [==============================] - 20s 782ms/step - loss: 0.4288 - accuracy: 0.8029 - val_loss: 0.3958 - val_accuracy: 0.8780


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.597324,0.659255,0.521951,0.661648
1,1,0.727494,0.582978,0.731707,0.547152
2,2,0.733577,0.533828,0.682927,0.560866
3,3,0.783455,0.470401,0.790244,0.420401
4,4,0.802920,0.428887,0.878049,0.395796


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6780 - accuracy: 0.5827 - val_loss: 0.7077 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 784ms/step - loss: 0.6612 - accuracy: 0.5681 - val_loss: 0.6706 - val_accuracy: 0.5415

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6490 - accuracy: 0.6338 - val_loss: 0.6663 - val_accuracy: 0.5561

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 771ms/step - loss: 0.6441 - accuracy: 0.6642 - val_loss: 0.6645 - val_accuracy: 0.5610

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 792ms/step - loss: 0.6412

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 843ms/step - loss: 0.6724 - accuracy: 0.5730 - val_loss: 0.6491 - val_accuracy: 0.6537
Epoch 2/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6191 - accuracy: 0.7375
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 21s 791ms/step - loss: 0.6151 - accuracy: 0.7397 - val_loss: 0.7671 - val_accuracy: 0.4341
Epoch 3/50
26/26 [==============================] - 20s 785ms/step - loss: 0.5791 - accuracy: 0.7628 - val_loss: 0.5850 - val_accuracy: 0.7902
Epoch 4/50
26/26 [==============================] - 20s 785ms/step - loss: 0.5497 - accuracy: 0.8212 - val_loss: 0.5494 - val_accuracy: 0.8049
Epoch 5/50
26/26 [==============================] - 21s 792ms/step - loss: 0.5348 - accuracy: 0.8358 - val_loss: 0.5362 - val_accuracy: 0.8244
Epoch 6/50
26/26 [==============================] - 21s 796ms/step - loss: 0.5149 - accuracy: 0.8333 - val_loss: 0.5158 - 

In [ ]:
%tensorboard --logdir log_dir